# Hockey Statistics Scraper
Dieses Notebook zeigt, wie man Hockey-Statistiken von einer Website scrapt.

In [23]:
# Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

print("✅ Alle Bibliotheken erfolgreich importiert!")

✅ Alle Bibliotheken erfolgreich importiert!


In [24]:
def scrape_hockey_table(url_or_html):
    """
    Scrapt Hockey-Statistiken aus einer HTML-Tabelle
    
    Args:
        url_or_html (str): URL der Webseite oder HTML-String
        
    Returns:
        pandas.DataFrame: Gescrapte Team-Daten
    """
    
    # 1. Content laden (URL oder HTML-String)
    if url_or_html.startswith('http'):
        # Von URL laden
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        response = requests.get(url_or_html, headers=headers)
        html_content = response.content
    else:
        # Direkter HTML-String
        html_content = url_or_html
    
    # 2. HTML parsen
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', class_='table')
    
    if not table:
        raise ValueError("❌ Keine Tabelle mit class='table' gefunden!")
    
    # 3. Header extrahieren
    headers = [th.get_text().strip() for th in table.find('tr').find_all('th')]
    
    # 4. Team-Daten extrahieren (class="team")
    data_rows = []
    for row in table.find_all('tr', class_='team'):
        row_data = [td.get_text().strip() or None for td in row.find_all('td')]
        data_rows.append(row_data)
    
    # 5. DataFrame erstellen und numerische Spalten konvertieren
    df = pd.DataFrame(data_rows, columns=headers)
    
    # 6. Numerische Spalten konvertieren
    numeric_cols = ['Year', 'Wins', 'Losses', 'Goals For (GF)', 'Goals Against (GA)', 'Win %', '+ / -']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    print(f"✅ {len(df)} Teams erfolgreich gescrapt!")
    return df

In [25]:
# Test mit echter URL (erste Seite)
url = "https://www.scrapethissite.com/pages/forms/?page_num=1"
print(f"🔍 Scrape URL: {url}")

try:
    df = scrape_hockey_table(url)
    print(f"\n📊 Erste 5 Teams:")
    print(df.head())
except Exception as e:
    print(f"❌ Fehler: {e}")

🔍 Scrape URL: https://www.scrapethissite.com/pages/forms/?page_num=1
✅ 25 Teams erfolgreich gescrapt!

📊 Erste 5 Teams:
            Team Name  Year  Wins  Losses OT Losses  Win %  Goals For (GF)  \
0       Boston Bruins  1990    44      24      None  0.550             299   
1      Buffalo Sabres  1990    31      30      None  0.388             292   
2      Calgary Flames  1990    46      26      None  0.575             344   
3  Chicago Blackhawks  1990    49      23      None  0.613             284   
4   Detroit Red Wings  1990    34      38      None  0.425             273   

   Goals Against (GA)  + / -  
0                 264     35  
1                 278     14  
2                 263     81  
3                 211     73  
4                 298    -25  
✅ 25 Teams erfolgreich gescrapt!

📊 Erste 5 Teams:
            Team Name  Year  Wins  Losses OT Losses  Win %  Goals For (GF)  \
0       Boston Bruins  1990    44      24      None  0.550             299   
1      Buffalo Sab

In [26]:
# Test mit HTML-String (Beispiel)
html_string = '''
<table class="table">
<tbody>
<tr>
    <th>Team Name</th>
    <th>Year</th>
    <th>Wins</th>
    <th>Losses</th>
    <th>OT Losses</th>
    <th>Win %</th>
    <th>Goals For (GF)</th>
    <th>Goals Against (GA)</th>
    <th>+ / -</th>
</tr>
<tr class="team">
    <td class="name">Boston Bruins</td>
    <td class="year">1990</td>
    <td class="wins">44</td>
    <td class="losses">24</td>
    <td class="ot-losses"></td>
    <td class="pct text-success">0.55</td>
    <td class="gf">299</td>
    <td class="ga">264</td>
    <td class="diff text-success">35</td>
</tr>
</tbody>
</table>
'''

print("🧪 Test mit HTML-String:")
df_html = scrape_hockey_table(html_string)
print(df_html)

🧪 Test mit HTML-String:
✅ 1 Teams erfolgreich gescrapt!
       Team Name  Year  Wins  Losses OT Losses  Win %  Goals For (GF)  \
0  Boston Bruins  1990    44      24      None   0.55             299   

   Goals Against (GA)  + / -  
0                 264     35  


In [27]:
# Daten speichern (wenn vorhanden)
if 'df' in locals() and not df.empty:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'hockey_data_{timestamp}.csv'
    df.to_csv(filename, index=False)
    print(f"💾 Daten gespeichert als: {filename}")
    
    # Statistiken anzeigen
    print(f"\n📈 STATISTIKEN:")
    print(f"Anzahl Teams: {len(df)}")
    if 'Year' in df.columns:
        print(f"Jahre: {df['Year'].min()} - {df['Year'].max()}")
    if 'Team Name' in df.columns:
        print(f"Verschiedene Teams: {df['Team Name'].nunique()}")
else:
    print("⚠️  Kein DataFrame zum Speichern verfügbar. Führen Sie zuerst die Scraping-Zellen aus!")

💾 Daten gespeichert als: hockey_data_20251013_161533.csv

📈 STATISTIKEN:
Anzahl Teams: 25
Jahre: 1990 - 1991
Verschiedene Teams: 21


## Multi-Page Scraping
Scrape alle Seiten der Hockey-Statistiken (1-24 Seiten)

In [28]:
def scrape_all_hockey_pages(max_pages=24, delay=1.5):
    """
    Scrapt alle Seiten der Hockey-Statistiken
    
    Args:
        max_pages (int): Maximale Anzahl Seiten (Standard: 24)
        delay (float): Verzögerung zwischen Requests in Sekunden
        
    Returns:
        pandas.DataFrame: Alle gescrapten Team-Daten
    """
    
    print(f"🚀 Scrape {max_pages} Seiten Hockey-Daten...")
    all_data = []
    headers_list = []
    
    for page in range(1, max_pages + 1):
        url = f"https://www.scrapethissite.com/pages/forms/?page_num={page}"
        print(f"📡 Seite {page}/{max_pages}")
        
        try:
            # Request senden
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            # HTML parsen
            soup = BeautifulSoup(response.content, 'html.parser')
            table = soup.find('table', class_='table')
            
            if table:
                # Header (nur bei Seite 1)
                if page == 1:
                    headers_list = [th.get_text().strip() for th in table.find('tr').find_all('th')]
                
                # Team-Daten extrahieren
                team_rows = table.find_all('tr', class_='team')
                
                for row in team_rows:
                    row_data = [td.get_text().strip() or None for td in row.find_all('td')]
                    if len(row_data) == len(headers_list):
                        all_data.append(row_data)
                
                print(f"   ✅ {len(team_rows)} Teams")
            else:
                print(f"   ⚠️  Keine Tabelle")
            
            # Höfliche Verzögerung
            if page < max_pages:
                time.sleep(delay)
                
        except Exception as e:
            print(f"   ❌ Fehler: {e}")
    
    # DataFrame erstellen
    if all_data:
        df = pd.DataFrame(all_data, columns=headers_list)
        
        # Datentypen optimieren
        numeric_cols = ['Year', 'Wins', 'Losses', 'Goals For (GF)', 'Goals Against (GA)', '+ / -']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        if 'Win %' in df.columns:
            df['Win %'] = pd.to_numeric(df['Win %'], errors='coerce')
        
        print(f"✅ Gesamt: {len(df)} Teams von {max_pages} Seiten gescrapt!")
        return df
    else:
        print("❌ Keine Daten gefunden!")
        return pd.DataFrame()

In [30]:
# Beispiel: Scrape nur die ersten 3 Seiten (zum Testen)
print("🧪 Test mit nur 3 Seiten:")
df_multi = scrape_all_hockey_pages(max_pages=24, delay=1.0)

if not df_multi.empty:
    print(f"\n📊 ERGEBNISSE:")
    print(f"Teams gesamt: {len(df_multi)}")
    print(f"Jahre: {df_multi['Year'].min()} - {df_multi['Year'].max()}")
    print(f"Verschiedene Teams: {df_multi['Team Name'].nunique()}")
    
    print(f"\n🏆 TOP 5 TEAMS (meiste Siege):")
    top5 = df_multi.nlargest(5, 'Wins')[['Team Name', 'Year', 'Wins', 'Losses']]
    print(top5)

🧪 Test mit nur 3 Seiten:
🚀 Scrape 24 Seiten Hockey-Daten...
📡 Seite 1/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 2/24
📡 Seite 2/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 3/24
📡 Seite 3/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 4/24
📡 Seite 4/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 5/24
📡 Seite 5/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 6/24
📡 Seite 6/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 7/24
📡 Seite 7/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 8/24
📡 Seite 8/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 9/24
📡 Seite 9/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 10/24
📡 Seite 10/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 11/24
📡 Seite 11/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 12/24
📡 Seite 12/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 13/24
📡 Seite 13/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 14/24
📡 Seite 14/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 15/24
📡 Seite 15/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 16/24
📡 Seite 16/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 17/24
📡 Seite 17/24
   ✅ 25 Teams
   ✅ 25 Teams
📡 Seite 18/24
📡 Sei

In [31]:
# Hier kannst du eigenen Code ergänzen oder weitere Tests durchführen
# Beispiel: Zeige die ersten 10 Teams aus dem Multi-Page-Scraping
if 'df_multi' in locals() and not df_multi.empty:
    print("\n📋 Die ersten 10 Teams aus allen Seiten:")
    print(df_multi.head(10))


📋 Die ersten 10 Teams aus allen Seiten:
               Team Name  Year  Wins  Losses OT Losses  Win %  Goals For (GF)  \
0          Boston Bruins  1990    44      24      None  0.550             299   
1         Buffalo Sabres  1990    31      30      None  0.388             292   
2         Calgary Flames  1990    46      26      None  0.575             344   
3     Chicago Blackhawks  1990    49      23      None  0.613             284   
4      Detroit Red Wings  1990    34      38      None  0.425             273   
5        Edmonton Oilers  1990    37      37      None  0.463             272   
6       Hartford Whalers  1990    31      38      None  0.388             238   
7      Los Angeles Kings  1990    46      24      None  0.575             340   
8  Minnesota North Stars  1990    27      39      None  0.338             256   
9     Montreal Canadiens  1990    39      30      None  0.487             273   

   Goals Against (GA)  + / -  
0                 264     35  
1    

## Komplette Webscraping-Checkliste & Workflow
Hier sind alle Aufgaben und der Workflow für das Hockey-Webscraping in einem Notebook vereint:

1. CSV speichern
2. CSV laden
3. Letzte Seite erkennen
4. Alle Seiten scrapen
5. Daten analysieren

In [ ]:
# 1. CSV speichern
import pandas as pd
from datetime import datetime
data = {
    'Team Name': ['Boston Bruins', 'Chicago Blackhawks'],
    'Year': [1990, 1991],
    'Wins': [44, 33]
}
df = pd.DataFrame(data)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f'hockey_teams_{timestamp}.csv'
df.to_csv(csv_filename, index=False)
print(f'✅ CSV gespeichert als: {csv_filename}')

# 2. CSV laden
df_loaded = pd.read_csv(csv_filename)
print('📂 Geladene Daten:')
print(df_loaded)

# 3. Letzte Seite erkennen
import requests
from bs4 import BeautifulSoup
def is_last_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', class_='table')
    if not table:
        return True
    teams = table.find_all('tr', class_='team')
    return len(teams) == 0
test_url = 'https://www.scrapethissite.com/pages/forms/?page_num=25'
if is_last_page(test_url):
    print('🚩 Letzte Seite erreicht!')
else:
    print('➡️ Weitere Seiten vorhanden.')

# 4. Alle Seiten scrapen
import time
def scrape_all_hockey_pages(max_pages=25, delay=1.0):
    all_data = []
    headers_list = []
    for page in range(1, max_pages + 1):
        url = f'https://www.scrapethissite.com/pages/forms/?page_num={page}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', class_='table')
        if not table:
            print(f'❌ Keine Tabelle auf Seite {page}')
            break
        if page == 1:
            headers_list = [th.get_text().strip() for th in table.find('tr').find_all('th')]
        team_rows = table.find_all('tr', class_='team')
        for row in team_rows:
            row_data = [td.get_text().strip() or None for td in row.find_all('td')]
            if len(row_data) == len(headers_list):
                all_data.append(row_data)
        print(f'✅ Seite {page}: {len(team_rows)} Teams')
        time.sleep(delay)
    df_multi = pd.DataFrame(all_data, columns=headers_list)
    return df_multi
df_multi = scrape_all_hockey_pages(max_pages=25, delay=1.0)  # Zum Testen nur 3 Seiten
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
multi_csv_filename = f'hockey_all_teams_{timestamp}.csv'
df_multi.to_csv(multi_csv_filename, index=False)
print(f'💾 Alle Teams gespeichert als: {multi_csv_filename}')

# 5. Daten analysieren
if not df_multi.empty:
    print('📋 Die ersten 10 Teams aus allen Seiten:')
    print(df_multi.head(10))

✅ CSV gespeichert als: hockey_teams_20251013_161957.csv
📂 Geladene Daten:
            Team Name  Year  Wins
0       Boston Bruins  1990    44
1  Chicago Blackhawks  1991    33
🚩 Letzte Seite erreicht!
🚩 Letzte Seite erreicht!
✅ Seite 1: 25 Teams
✅ Seite 1: 25 Teams
✅ Seite 2: 25 Teams
✅ Seite 2: 25 Teams
✅ Seite 3: 25 Teams
✅ Seite 3: 25 Teams
💾 Alle Teams gespeichert als: hockey_all_teams_20251013_162004.csv
📋 Die ersten 10 Teams aus allen Seiten:
               Team Name  Year Wins Losses OT Losses  Win % Goals For (GF)  \
0          Boston Bruins  1990   44     24      None   0.55            299   
1         Buffalo Sabres  1990   31     30      None  0.388            292   
2         Calgary Flames  1990   46     26      None  0.575            344   
3     Chicago Blackhawks  1990   49     23      None  0.613            284   
4      Detroit Red Wings  1990   34     38      None  0.425            273   
5        Edmonton Oilers  1990   37     37      None  0.463            272   
6